In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import time

import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score

import random, os, json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, GRU, Dropout, Dense
from tensorflow.keras import backend as K

import sys
sys.path.append("../../../")
from rnns_architectures.utils import *

from joblib import Parallel, delayed
import multiprocessing

import pickle

# FUNCTIONS OF THE MODEL

In [ ]:
def single_label(df):
    """Groups the DataFrame by 'Admissiondboid' and assigns a single label. 
    If any patient has a day with 'individualMRGerm' equal to 1 (MDR), 
    is labeled as 1; otherwise, it is labeled as 0 (Non-MDR).
    """
    
    grouped = df.groupby('Admissiondboid')
    results = []
    
    for admissionboid, group in grouped:
        if (group['individualMRGerm'] == 1).any():
            results.append({'Admissiondboid': admissionboid, 'individualMRGerm': 1})
        else:
            results.append({'Admissiondboid': admissionboid, 'individualMRGerm': 0})

    results_df = pd.DataFrame(results)

    return results_df

def weighted_binary_crossentropy(hyperparameters):
    w1 = hyperparameters["w1"]
    w2 = hyperparameters["w2"]
    """
    Binary form of weighted binary cross entropy.
      WBCE(p_t) = -w * (1 - p_t)* log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    Usage:
     model.compile(loss=[weighted_binary_crossentropyv2(hyperparameters)], metrics=["accuracy"], optimizer=adam)
    """
    def loss(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(w1 * (1. - pt_1) * K.log(pt_1)) \
               -K.sum(w2 * (pt_0) * K.log(1. - pt_0))

    return loss

def build_model(hyperparameters):
    """
    Builds a GRU model based on several hyperparameters.

    Args:
        - hyperparameters: Dictionary containing the hyperparameters. 
    Returns:
        - model: A tf.keras.Model with the compiled model.
    """
    
    dynamic_input = tf.keras.layers.Input(shape=(hyperparameters["n_time_steps"], hyperparameters["layers"][0]))
    masked = tf.keras.layers.Masking(mask_value=hyperparameters['mask_value'])(dynamic_input)

    gru_encoder = tf.keras.layers.GRU(
        hyperparameters["layers"][1],
        dropout=hyperparameters['dropout'],
        return_sequences=False,
        activation=hyperparameters['activation'],
        use_bias=False
    )(masked)

    output = tf.keras.layers.Dense(1, use_bias=False, activation="sigmoid")(gru_encoder)

    model = tf.keras.Model(dynamic_input, [output])
    model.compile(
        loss=weighted_binary_crossentropy(hyperparameters),
        optimizer=tf.keras.optimizers.Adam(learning_rate=hyperparameters["lr_scheduler"]),
        metrics=['accuracy', "AUC"],
        weighted_metrics = []
    )
        
    return model


def run_network(X_train, X_val, y_train, y_val, hyperparameters, seed):
    """
    Trains and evaluates the built GRU model based on the provided data and hyperparameters.

    Args:
        - X_train, X_val, y_train, y_val: numpy.ndarray. Training (T) and Validation (V) data labels.
        - sample_weights_train, sample_weights_val: numpy.ndarray. Weights for the T and V data to handle class imbalance.
        - hyperparameters: Dictionary containing the hyperparameters.
        - seed: Integer seed for reproducibility.
    Returns:
        - model: A tf.keras.Model with the trained model.
        - hist:  The training history.
        - earlystopping: The early stopping callback.
    """
    batch_size = hyperparameters['batch_size']
    n_epochs_max = hyperparameters['n_epochs_max']

    model = None
    model = build_model(hyperparameters)
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  min_delta=hyperparameters["mindelta"],
                                                  patience=hyperparameters["patience"],
                                                  restore_best_weights=True,
                                                  mode="min")
    hist = model.fit(X_train, y_train,
                     validation_data=(X_val, y_val),
                     callbacks=[earlystopping], batch_size=batch_size, epochs=n_epochs_max,
                     verbose=hyperparameters['verbose'])
    
    return model, hist, earlystopping


def calculate_metrics(model, X_test, y_test):

    y_pred_test = model.predict(X_test)
    
    # Calculate metrics
    accuracy_test = sklearn.metrics.accuracy_score(y_test.individualMRGerm.values, np.round(y_pred_test))
    tn, fp, fn, tp = confusion_matrix(y_test.individualMRGerm.values, np.round(y_pred_test)).ravel()
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = (2 * recall * precision) / (recall + precision)
    roc_auc = sklearn.metrics.roc_auc_score(y_test.individualMRGerm.values, y_pred_test)
    
    # Dataframe
    df_metrics = pd.DataFrame({
        'accuracy': [accuracy_test],
        'specificity': [specificity],
        'recall': [recall],
        'precision': [precision],
        'f1_score': [f1_score],
        'roc_auc': [roc_auc]
    })
    return df_metrics

In [ ]:
import gc
from tensorflow.keras import backend as K

import tensorflow as tf
import gc
from tensorflow.keras import backend as K

def evaluate_combination(k, l, m, b, hyperparameters, dropout, layers, lr_scheduler, adjustment_factor, activation, seed, split, norm, n_time_steps):
    hyperparameters_copy = hyperparameters.copy()
    hyperparameters_copy['dropout'] = dropout[k]
    hyperparameters_copy['layers'] = layers[l]
    hyperparameters_copy['lr_scheduler'] = lr_scheduler[m]
    hyperparameters_copy['adjustment_factor'] = adjustment_factor[0]
    hyperparameters_copy['activation'] = activation[b]
    
    v_val_loss = []
    data_paths = []  # Para guardar las rutas de los archivos

    for i in range(5):
        # Guardar las rutas a los archivos sin cargar los datos completos en memoria
        X_train_path = f"../../../DATA/MDR/{split}/X_train_tensor_{i}{norm}.npy"
        y_train_path = f"../../../DATA/MDR/{split}/y_train_tensor_{i}{norm}.csv"
        X_val_path = f"../../../DATA/MDR/{split}/X_val_tensor_{i}{norm}.npy"
        y_val_path = f"../../../DATA/MDR/{split}/y_val_tensor_{i}{norm}.csv"

        data_paths.append((X_train_path, y_train_path, X_val_path, y_val_path))

        X_train = np.load(X_train_path)
        y_train = pd.read_csv(y_train_path)
        y_train = single_label(y_train)
        X_val = np.load(X_val_path)
        y_val = pd.read_csv(y_val_path)
        y_val = single_label(y_val)

        gc.collect()

        reset_keras()

        model, hist, early = run_network(
            X_train, X_val,
            y_train.individualMRGerm.values, 
            y_val.individualMRGerm.values,
            hyperparameters_copy,
            seed
        )

        v_val_loss.append(np.min(hist.history["val_loss"]))

        del X_train, y_train, X_val, y_val, model, hist, early
        K.clear_session() 
        gc.collect()  

    metric_dev = np.mean(v_val_loss)
    gc.collect()
    
    return (metric_dev, k, l, m, b, data_paths)



def myCVGridParallel(hyperparameters, dropout, lr_scheduler, layers, adjustment_factor, activation, seed, split, norm, n_time_steps=14):
    bestHyperparameters = {}
    bestMetricDev = np.inf
    best_data_paths = [] 

    results = Parallel(n_jobs=12)(
        delayed(evaluate_combination)(k, l, m, b, hyperparameters, dropout, layers, lr_scheduler, adjustment_factor, activation, seed, split, norm, n_time_steps)
        for k in range(len(dropout))
        for l in range(len(layers))
        for m in range(len(lr_scheduler))
        for b in range(len(activation))
    )

    for metric_dev, k, l, m, b, data_paths in results:
        if metric_dev < bestMetricDev:
            bestMetricDev = metric_dev
            bestHyperparameters = {
                'dropout': dropout[k],
                'layers': layers[l],
                'lr_scheduler': lr_scheduler[m],
                'adjustment_factor': adjustment_factor[0],
                'activation': activation[b]
            }
            best_data_paths = data_paths
            
    return bestHyperparameters, best_data_paths

# HYPERPARAMETERS

- **seeds**: Seed values to ensure reproducibility.
- **input_shape**: Number of features in each time step of the input data.
- **n_time_steps**: Number of time steps in the input sequence.
- **batch_size**: Number of batches for training.
- **n_epochs_max**: Maximum number of epochs for training.
- **layer_list**: A list with different configurations for the layers of the model.
- **dropout**: Dropout rates.
- **lr_scheduler**: Learning rates.
- **norm**: Type of normalization applied to the data.


In [ ]:
seeds = [9, 18, 35]

input_shape = 71
n_time_steps = 14
batch_size = 32
n_epochs_max = 1000

layer_list = [
    [input_shape, 3, 1], [input_shape, 5, 1], [input_shape, 10, 1],
    [input_shape, 20, 1],  [input_shape, 30, 1], [input_shape, 40, 1], 
    [input_shape, 50, 1], [input_shape, 60, 1]
]

dropout = [0.0, 0.15, 0.3]
lr_scheduler = [1e-1, 1e-2, 1e-3, 1e-4]


adjustment_factor = [1]  


activation = ['tanh', 'LeakyReLU']
 
norm = "robustNorm"

w2 = 0.18
w1 = 0.82

hyperparameters = {
    "n_time_steps": n_time_steps,
    "mask_value": 666,
    "w1":w1, "w2":w2, 
    "batch_size": batch_size,
    "n_epochs_max": n_epochs_max,
    "monitor": "val_loss",
    "mindelta": 0,
    "patience": 50,
    "dropout": 0.0,
    "verbose": 0,
}

# PREDICTIONS

In [ ]:
import os
import pickle
import time
import numpy as np
import pandas as pd
  

run_model = True 
if run_model:
    loss_train = []
    loss_dev = []
    v_models = []

    bestHyperparameters_bySplit = {}
    y_pred_by_split = {}

    for i in [1, 2, 3]:
        init = time.time()
        # LOAD TEST AND PRE-TRAIN
        X_test = np.load("../../../DATA/MDR/s" + str(i) + "/X_test_tensor_" + norm + ".npy")
        y_test = pd.read_csv("../../../DATA/MDR/s" + str(i) + "/y_test_tensor_" + norm + ".csv")
        y_test = single_label(y_test)
        #GridSearch of hyperparameters 
        bestHyperparameters, best_data_paths = myCVGridParallel(hyperparameters,
                                                               dropout,
                                                               lr_scheduler,
                                                               layer_list,
                                                               adjustment_factor,
                                                               activation,
                                                               seeds[i-1],
                                                               "s"+str(i),
                                                               norm)
        fin = time.time()

        X_train_path, y_train_path, X_val_path, y_val_path = best_data_paths[0] 
    
        # Cargar los datos desde las rutas
        X_train = np.load(X_train_path)
        y_train = pd.read_csv(y_train_path)
        y_train = single_label(y_train)
        X_val = np.load(X_val_path)
        y_val = pd.read_csv(y_val_path)
        y_val = single_label(y_val)

        bestHyperparameters_bySplit[str(i)] = bestHyperparameters

        # Save best hyperparameters for current split
        split_directory = './Results_GRU/split_' + str(i)
        if not os.path.exists(split_directory):
            os.makedirs(split_directory)

        with open(os.path.join(split_directory, f"bestHyperparameters_split_{i}.pkl"), 'wb') as f:
            pickle.dump(bestHyperparameters, f)

        hyperparameters = {
            'n_time_steps': hyperparameters["n_time_steps"],
            'mask_value': hyperparameters["mask_value"],
            "w1":hyperparameters["w1"], "w2":hyperparameters["w2"],
            'batch_size': hyperparameters["batch_size"],
            'n_epochs_max': hyperparameters["n_epochs_max"],
            'monitor':  hyperparameters["monitor"],
            "mindelta": hyperparameters["mindelta"],
            "patience": hyperparameters["patience"],
            "dropout": bestHyperparameters["dropout"],
            "layers": bestHyperparameters["layers"],
            "lr_scheduler": bestHyperparameters["lr_scheduler"],
            "adjustment_factor": bestHyperparameters["adjustment_factor"],
            "activation": bestHyperparameters["activation"],
            'verbose': 0
        }

        # --- TRY ON TEST -----------------------------------------------------------------------

        reset_keras()

        model, hist, early = run_network(
            X_train, 
            X_val,
            y_train.individualMRGerm.values, 
            y_val.individualMRGerm.values,
            hyperparameters,
            seeds[i-1]
        )

        v_models.append(model)
        loss_train.append(hist.history['loss'])
        loss_dev.append(hist.history['val_loss'])
        
        #Calculate metrics and predictions
        y_pred = model.predict(x=X_test)
        y_pred_by_split[str(i)] = y_pred
        
        metrics = calculate_metrics(model, X_test, y_test)

        # Save y_pred and metrics for current split
        with open(os.path.join(split_directory, f"y_pred_split_{i}.pkl"), 'wb') as f:
            pickle.dump(y_pred, f)
        with open(os.path.join(split_directory, f"model_split_{i}.h5"), 'wb') as f:
            pickle.dump(model, f)
        with open(os.path.join(split_directory, f"metrics_{i}.pkl"), 'wb') as f:
            pickle.dump(metrics, f)


    # # END EXECUTION - SAVE AGGREGATED RESULTS
    print('END')

In [2]:
directory = './Results_GRU'
def load_from_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

y_pred_by_split = {}

y_pred_by_split_1 = load_from_pickle(os.path.join('./Results_GRU/split_1', "metrics_1.pkl"))
y_pred_by_split_2 = load_from_pickle(os.path.join('./Results_GRU/split_2', "metrics_2.pkl"))
y_pred_by_split_3 = load_from_pickle(os.path.join('./Results_GRU/split_3', "metrics_3.pkl"))

y_pred_by_split['1'] = y_pred_by_split_1
y_pred_by_split['2'] = y_pred_by_split_2
y_pred_by_split['3'] = y_pred_by_split_3

In [3]:
y_pred_by_split_1

,accuracy,specificity,recall,precision,f1_score,roc_auc
0,0.794286,0.822346,0.632258,0.381323,0.475728,0.793231


In [4]:
y_pred_by_split_2

,accuracy,specificity,recall,precision,f1_score,roc_auc
0,0.779048,0.804299,0.644578,0.382143,0.479821,0.800789


In [5]:
y_pred_by_split_3

,accuracy,specificity,recall,precision,f1_score,roc_auc
0,0.551429,0.571101,0.455056,0.178022,0.255924,0.557214
